## IMDB Movie Review Sentiment Analysis

In [1]:
import re, string, unicodedata
# import contractions
# import inflect
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer, WordNetLemmatizer

def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_links_characters(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub('\[[^]]*\]', '', text)
    return text

def regular_preprocess(text):
    text = remove_html(text)
    text = remove_links_characters(text)
    return text

def remove_stopwords(docs, stopwords):
    docs_ref = []
    for doc in docs:
        word_list = doc.lower().split()
        word_list_ref = [word for word in word_list if word not in stopwords]
        word_str_ref = ' '.join(word_list_ref)
        docs_ref.append(word_str_ref)
    return docs_ref

def stem_words(docs):
    stemmer = PorterStemmer()
    stems = []
    for doc in docs:
        word_list = doc.lower().split()
        for word in word_list:
            stem = stemmer.stem(word)
            stems.append(stem)
        stems_str = ' '.join(stems)
        stems.append(stems_str)
    return stems_str

def preprocess(data):
    refined_data = []
    for dp in data:
        refined_data.append(regular_preprocess(dp))        
    return refined_data

In [2]:
import glob
import os
import csv
import numpy as np
from preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from scipy.stats import uniform
from sklearn.model_selection import GridSearchCV

In [3]:
# copy contents of all files in both folders into a list
train_data = []
test_data = []

# train data
train_neg = glob.glob(os.path.join(os.getcwd(), "Dataset/train/neg", "*.txt"))
for f_path in train_neg:
    with open(f_path) as f:
        train_data.append(f.read())

train_pos = glob.glob(os.path.join(os.getcwd(), "Dataset/train/pos", "*.txt"))
for f_path in train_pos:
    with open(f_path) as f:
        train_data.append(f.read())

def sort_nicely(l):
# Sort the given list in the way that humans expect.
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    l.sort(key=alphanum_key)

test_files = glob.glob(os.path.join(os.getcwd(), "Dataset/test", "*.txt"))
sort_nicely(test_files)
test_files_ids = [int(re.sub("[^0-9]","", item)) for item in test_files]

for f_path in test_files:
    with open(f_path) as f:
        test_data.append(f.read())

In [4]:
# targets: first 12500 are pos, next 12500 are neg
targets = [0 if i<12500 else 1 for i in range(25000)]

train_data_clean = preprocess(train_data)
test_data_clean = preprocess(test_data)

# train_data_clean = remove_stopwords(train_data_clean, stopwords)
# test_data_clean = remove_stopwords(test_data_clean, stopwords)
# train_data_clean = stem_words(train_data_clean)
# test_data_clean = stem_words(test_data_clean) 

# save to a file if using above complex 

X_train, X_validation, y_train, y_validation = train_test_split(train_data_clean, targets, train_size=0.8, test_size=0.2, random_state=1)

def display_results(y_val, y_pred):
    print(metrics.classification_report(y_val, y_pred))
    print("Accuracy % = ", metrics.accuracy_score(y_val, y_pred))

## SGD

In [49]:
SGDpclf = Pipeline([
    ('vect', CountVectorizer(analyzer='word', max_df=1.0, min_df=0,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', SGDClassifier(alpha=1e-5, epsilon=0.1, random_state=1)),
])
         
SGDpclf.fit(X_train, y_train)
y_pred_sgd_val = SGDpclf.predict(X_validation)

y_pred_sgd_test = SGDpclf.predict(test_data_clean)
with open('submissionSGD.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(("Id", "Category"))
    writer.writerows(zip(test_files_ids, y_pred_sgd_test))

display_results(y_validation, y_pred_sgd_val)

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      2517
           1       0.90      0.92      0.91      2483

   micro avg       0.91      0.91      0.91      5000
   macro avg       0.91      0.91      0.91      5000
weighted avg       0.91      0.91      0.91      5000

Accuracy % =  0.9118


## Linear SVM

In [57]:
SVMpclf = Pipeline([
    ('vect', CountVectorizer(max_df=0.5, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', LinearSVC(C=0.5)),
])

SVMpclf.fit(X_train, y_train)
y_pred_svm_val = SVMpclf.predict(X_validation)

y_pred_svm_test = SVMpclf.predict(test_data_clean)
with open('submissionLSVM.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(("Id", "Category"))
    writer.writerows(zip(test_files_ids, y_pred_svm_test))

display_results(y_validation, y_pred_svm_val)

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      2517
           1       0.90      0.92      0.91      2483

   micro avg       0.91      0.91      0.91      5000
   macro avg       0.91      0.91      0.91      5000
weighted avg       0.91      0.91      0.91      5000

Accuracy % =  0.9108


## Logistic

In [50]:
LRpclf = Pipeline([
    ('vect', CountVectorizer(binary=True, max_df=0.5, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', LogisticRegression()),
])

LRpclf.fit(X_train, y_train)
y_pred_lr_val = LRpclf.predict(X_validation)

y_pred_lr_test = LRpclf.predict(test_data_clean)
with open('submissionLR.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(("Id", "Category"))
    writer.writerows(zip(test_files_ids, y_pred_lr_test))

display_results(y_validation, y_pred_lr_val)

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      2517
           1       0.89      0.91      0.90      2483

   micro avg       0.90      0.90      0.90      5000
   macro avg       0.90      0.90      0.90      5000
weighted avg       0.90      0.90      0.90      5000

Accuracy % =  0.8966


## Ensemble and use Major Vote

In [54]:
import pandas as pd
preds_val = pd.DataFrame()
preds_val['pred_val_1'] = y_pred_sgd_val
preds_val['pred_val_2'] = y_pred_svm_val
preds_val['pred_val_3'] = y_pred_lr_val

preds_val['maj_vote_val'] = preds_val.mode(axis = 1)

In [55]:
preds_val

,pred_val_1,pred_val_2,pred_val_3,maj_vote_val
0,1,0,0,0
1,1,1,0,1
2,1,1,1,1
3,1,1,1,1
4,0,0,0,0
5,0,0,0,0
6,1,1,1,1
7,1,1,1,1
8,1,1,1,1
9,0,0,0,0


In [58]:
preds = pd.DataFrame()
preds['pred1'] = y_pred_sgd_test
preds['pred2'] = y_pred_svm_test
preds['pred3'] = y_pred_lr_test

preds['maj_vote'] = preds.mode(axis = 1)

In [59]:
preds

,pred1,pred2,pred3,maj_vote
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,1,1,1,1
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [60]:
with open('submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(("Id", "Category"))
    writer.writerows(zip(test_files_ids, preds['maj_vote']))

In [ ]:
# def advanced_preprocess(text):
# #     words = nltk.word_tokenize(text)
#     words = replace_numbers(text)
#     words = remove_non_ascii(words)
#     words = to_lowercase(words)
#     words = remove_stopwords(words)
#     words = stem_words(words)
#     return words

# def replace_contractions(text):
#     """Replace contractions in string of text"""
#     return contractions.fix(text)
#
# def stem_words(words):
#     """Stem words in list of tokenized words"""
#     stemmer = PorterStemmer()
#     stems = []
#     for word in words:
#         stem = stemmer.stem(word)
#         stems.append(stem)
#     return stems
#
# def replace_numbers(words):
#     """Replace all interger occurrences in list of tokenized words with textual representation"""
#     p = inflect.engine()
#     new_words = []
#     for word in words:
#         if word.isdigit():
#             new_word = p.number_to_words(word)
#             new_words.append(new_word)
#         else:
#             new_words.append(word)
#     return new_words
#
# def lemmatize_verbs(words):
#     """Lemmatize verbs in list of tokenized words"""
#     lemmatizer = WordNetLemmatizer()
#     lemmas = []
#     for word in words:
#         lemma = lemmatizer.lemmatize(word, pos='v')
#         lemmas.append(lemma)
#     return lemmas
#
# def to_lowercase(words):
#     """Convert all characters to lowercase from list of tokenized words"""
#     new_words = []
#     for word in words:
#         new_word = word.lower()
#         new_words.append(new_word)
#     return new_words
#
# def remove_non_ascii(words):
#     """Remove non-ASCII characters from list of tokenized words"""
#     new_words = []
#     for word in words:
#         new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
#         new_words.append(new_word)
#     return new_words

### Only CountVectorizer(binary=True)

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

# Bag of Words vectorization
cv = CountVectorizer(binary=True).fit(X_train)
X_train_counts = cv.transform(X_train)
X_validation_counts = cv.transform(X_validation)

In [58]:
from sklearn.naive_bayes import MultinomialNB

clf_NB = MultinomialNB().fit(X_train_counts, y_train)
y_pred = clf_NB.predict(X_validation_counts)

# X_train_normalized = np.array(X_train_normalized)

# print(X_train_normalized.shape)
print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.83      0.85      2488
           1       0.84      0.88      0.86      2512

   micro avg       0.86      0.86      0.86      5000
   macro avg       0.86      0.86      0.86      5000
weighted avg       0.86      0.86      0.86      5000

Accuracy % =  0.8556


In [59]:
from sklearn.linear_model import LogisticRegression

clf_LR = LogisticRegression().fit(X_train_counts, y_train)
y_pred = clf_LR.predict(X_validation_counts)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.87      0.89      0.88      2488
           1       0.89      0.87      0.88      2512

   micro avg       0.88      0.88      0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000

Accuracy % =  0.8814


In [ ]:
# from sklearn.tree import DecisionTreeClassifier

# clf_DT = DecisionTreeClassifier().fit(X_train_counts, y_train)
# y_pred = clf_DT.predict(X_validation_counts)

# print(metrics.classification_report(y_validation, y_pred))
# print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

In [60]:
from sklearn.svm import LinearSVC

clf_SVM = LinearSVC().fit(X_train_counts, y_train)
y_pred = clf_SVM.predict(X_validation_counts)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87      2488
           1       0.87      0.86      0.87      2512

   micro avg       0.87      0.87      0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000

Accuracy % =  0.8656


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### CountVectorizer(binary=True) with tf-idf

In [61]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Normalizer
from sklearn import metrics

# Bag of Words vectorization
cv = CountVectorizer(binary=True).fit(X_train)
X_train_counts = cv.transform(X_train)
X_validation_counts = cv.transform(X_validation)

# tfidf
tfidf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)
X_validation_tfidf = tfidf_transformer.transform(X_validation_counts)

# normalization
normalizer_tranformer = Normalizer().fit(X=X_train_tfidf)
X_train_normalized = normalizer_tranformer.transform(X_train_tfidf)
X_validation_normalized = normalizer_tranformer.transform(X_validation_tfidf)

In [62]:
clf_NB = MultinomialNB().fit(X_train_normalized, y_train)
y_pred = clf_NB.predict(X_validation_normalized)

# X_train_normalized = np.array(X_train_normalized)

print(X_train_normalized.shape)
print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

(20000, 68499)
              precision    recall  f1-score   support

           0       0.88      0.85      0.86      2488
           1       0.85      0.89      0.87      2512

   micro avg       0.87      0.87      0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000

Accuracy % =  0.8654


In [63]:
from sklearn.linear_model import LogisticRegression

clf_LR = LogisticRegression().fit(X_train_normalized, y_train)
y_pred = clf_LR.predict(X_validation_normalized)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.88      0.91      0.90      2488
           1       0.91      0.88      0.89      2512

   micro avg       0.89      0.89      0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

Accuracy % =  0.8944


In [64]:
# from sklearn.tree import DecisionTreeClassifier

# clf_DT = DecisionTreeClassifier().fit(X_train_normalized, y_train)
# y_pred = clf_DT.predict(X_validation_normalized)

# print(metrics.classification_report(y_validation, y_pred))
# print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

In [65]:
from sklearn.svm import LinearSVC

clf_SVM = LinearSVC().fit(X_train_normalized, y_train)
y_pred = clf_SVM.predict(X_validation_normalized)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90      2488
           1       0.90      0.88      0.89      2512

   micro avg       0.89      0.89      0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

Accuracy % =  0.8946


### Only CountVectorizer()

In [66]:
# Bag of Words vectorization
cv = CountVectorizer().fit(X_train)
X_train_counts = cv.transform(X_train)
X_validation_counts = cv.transform(X_validation)

In [67]:
clf_NB = MultinomialNB().fit(X_train_counts, y_train)
y_pred = clf_NB.predict(X_validation_counts)

# X_train_normalized = np.array(X_train_normalized)

# print(X_train_normalized.shape)
print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.81      0.84      2488
           1       0.83      0.88      0.85      2512

   micro avg       0.85      0.85      0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000

Accuracy % =  0.846


In [68]:
clf_LR = LogisticRegression().fit(X_train_counts, y_train)
y_pred = clf_LR.predict(X_validation_counts)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.88      0.90      0.89      2488
           1       0.90      0.88      0.89      2512

   micro avg       0.89      0.89      0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

Accuracy % =  0.8896


In [69]:
# clf_DT = DecisionTreeClassifier().fit(X_train_counts, y_train)
# y_pred = clf_DT.predict(X_validation_counts)

# print(metrics.classification_report(y_validation, y_pred))
# print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

In [70]:
clf_SVM = LinearSVC().fit(X_train_counts, y_train)
y_pred = clf_SVM.predict(X_validation_counts)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      2488
           1       0.88      0.86      0.87      2512

   micro avg       0.87      0.87      0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000

Accuracy % =  0.8686


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### CountVectorizer() with tf-idf

In [32]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

# Bag of Words vectorization
cv = CountVectorizer().fit(X_train)
X_train_counts = cv.transform(X_train)
X_validation_counts = cv.transform(X_validation)

# tfidf
tfidf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)
X_validation_tfidf = tfidf_transformer.transform(X_validation_counts)

# normalization
normalizer_tranformer = Normalizer().fit(X=X_train_tfidf)
X_train_normalized = normalizer_tranformer.transform(X_train_tfidf)
X_validation_normalized = normalizer_tranformer.transform(X_validation_tfidf)

In [34]:
from sklearn.naive_bayes import MultinomialNB
clf_NB = MultinomialNB().fit(X_train_normalized, y_train)
y_pred = clf_NB.predict(X_validation_normalized)

# X_train_normalized = np.array(X_train_normalized)

print(X_train_normalized.shape)
print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

(20000, 68354)
              precision    recall  f1-score   support

           0       0.85      0.89      0.87      2517
           1       0.88      0.84      0.86      2483

   micro avg       0.87      0.87      0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000

Accuracy % =  0.866


In [35]:
from sklearn.linear_model import LogisticRegression
clf_LR = LogisticRegression().fit(X_train_normalized, y_train)
y_pred = clf_LR.predict(X_validation_normalized)

print(metrics.classification_report(y_validation, y_pred))
print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.90      0.88      0.89      2517
           1       0.88      0.90      0.89      2483

   micro avg       0.89      0.89      0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

Accuracy % =  0.8944


In [75]:
# clf_DT = DecisionTreeClassifier().fit(X_train_normalized, y_train)
# y_pred = clf_DT.predict(X_validation_normalized)

# print(metrics.classification_report(y_validation, y_pred))
# print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

In [41]:
from sklearn.svm import LinearSVC
for c in [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.6, 0.55, 0.45, 1.0, 1.5, 0.525]:
    svm = LinearSVC(C=c)
    svm.fit(X_train_normalized, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, metrics.accuracy_score(y_validation, svm.predict(X_validation_normalized))))

# clf_SVM = LinearSVC().fit(X_train_normalized, y_train)
# y_pred = clf_SVM.predict(X_validation_normalized)

# print(metrics.classification_report(y_validation, y_pred))
# print("Accuracy % = ", metrics.accuracy_score(y_validation, y_pred))

Accuracy for C=0.001: 0.7858
Accuracy for C=0.005: 0.8366
Accuracy for C=0.01: 0.857
Accuracy for C=0.05: 0.8856
Accuracy for C=0.1: 0.8976
Accuracy for C=0.5: 0.899
Accuracy for C=0.6: 0.8974
Accuracy for C=0.55: 0.8986
Accuracy for C=0.45: 0.899
Accuracy for C=1.0: 0.8952
Accuracy for C=1.5: 0.8932
Accuracy for C=0.525: 0.8988
